## SIGNATE Assessment

In [26]:
import sys
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder

%matplotlib inline

import check_miss_value as chv

In [27]:
train_pd = pd.read_csv('./data/train.csv')
test_pd = pd.read_csv('./data/test.csv')

In [35]:
int(re.sub(r'\D', '', train_pd.host_response_rate[1]))
# re.sub(r'\D', '100%')
# train_pd.host_response_rate[1]
type(train_pd.host_response_rate)

pandas.core.series.Series

In [41]:
train_pd.host_response_rate.head().isnull()

0     True
1    False
2    False
3    False
4    False
Name: host_response_rate, dtype: bool

In [42]:
train_pd[train_pd['host_response_rate'].isnull()] = '0%'

In [44]:
train_pd.host_response_rate.head()

0      0%
1    100%
2    100%
3    100%
4    100%
Name: host_response_rate, dtype: object

In [46]:
train_pd['host_response_rate'] = train_pd.host_response_rate.apply(lambda x: re.sub(r'\D', '', x))
# train_pd['host_response_rate'] = train_pd.host_response_rate.apply(lambda x: x + '%')


In [47]:
train_pd.host_response_rate.head()

0      0
1    100
2    100
3    100
4    100
Name: host_response_rate, dtype: object

In [4]:
train_pd.describe()

,id,accommodates,bathrooms,bedrooms,beds,latitude,longitude,number_of_reviews,review_scores_rating,y
count,55583.000000,55583.000000,55436.000000,55512.000000,55487.000000,55583.000000,55583.000000,55583.000000,43027.000000,55583.000000
mean,27791.000000,3.152906,1.236426,1.265312,1.711770,38.450812,-92.342406,20.937283,94.077928,160.163647
std,16045.574343,2.153001,0.583074,0.849319,1.257822,3.080869,21.687844,37.951889,7.802154,168.087140
min,0.000000,1.000000,0.000000,0.000000,0.000000,33.338905,-122.511500,0.000000,20.000000,1.000000
25%,13895.500000,2.000000,1.000000,1.000000,1.000000,34.128205,-118.341876,1.000000,92.000000,74.000000
50%,27791.000000,2.000000,1.000000,1.000000,1.000000,40.663312,-76.995602,6.000000,96.000000,111.000000
75%,41686.500000,4.000000,1.000000,1.000000,2.000000,40.746146,-73.954703,23.000000,100.000000,185.000000
max,55582.000000,16.000000,8.000000,10.000000,18.000000,42.390437,-70.999166,605.000000,100.000000,1999.000000


In [6]:
test_pd.describe()

,id,accommodates,bathrooms,bedrooms,beds,latitude,longitude,number_of_reviews,review_scores_rating
count,18528.000000,18528.000000,18475.000000,18508.000000,18493.000000,18528.000000,18528.000000,18528.000000,14362.000000
mean,9263.500000,3.161863,1.231773,1.267236,1.708160,38.431396,-92.562881,20.790425,94.035719
std,5348.717229,2.155397,0.578944,0.860580,1.243063,3.078096,21.757419,37.457273,7.938919
min,0.000000,1.000000,0.000000,0.000000,0.000000,33.343011,-122.510940,0.000000,20.000000
25%,4631.750000,2.000000,1.000000,1.000000,1.000000,34.126672,-118.344098,1.000000,92.000000
50%,9263.500000,2.000000,1.000000,1.000000,1.000000,40.658229,-77.000323,6.000000,96.000000
75%,13895.250000,4.000000,1.000000,1.000000,2.000000,40.745958,-73.954432,23.000000,100.000000
max,18527.000000,16.000000,8.000000,10.000000,16.000000,42.390248,-70.985047,451.000000,100.000000


In [38]:
train_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55583 entries, 0 to 55582
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      55583 non-null  int64  
 1   accommodates            55583 non-null  int64  
 2   amenities               55583 non-null  object 
 3   bathrooms               55436 non-null  float64
 4   bed_type                55583 non-null  object 
 5   bedrooms                55512 non-null  float64
 6   beds                    55487 non-null  float64
 7   cancellation_policy     55583 non-null  object 
 8   city                    55583 non-null  object 
 9   cleaning_fee            55583 non-null  object 
 10  description             55583 non-null  object 
 11  first_review            43675 non-null  object 
 12  host_has_profile_pic    55435 non-null  object 
 13  host_identity_verified  55435 non-null  object 
 14  host_response_rate      41879 non-null

In [42]:
test_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18528 entries, 0 to 18527
Data columns (total 28 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      18528 non-null  int64  
 1   accommodates            18528 non-null  int64  
 2   amenities               18528 non-null  object 
 3   bathrooms               18475 non-null  float64
 4   bed_type                18528 non-null  object 
 5   bedrooms                18508 non-null  float64
 6   beds                    18493 non-null  float64
 7   cancellation_policy     18528 non-null  object 
 8   city                    18528 non-null  object 
 9   cleaning_fee            18528 non-null  object 
 10  description             18528 non-null  object 
 11  first_review            14572 non-null  object 
 12  host_has_profile_pic    18488 non-null  object 
 13  host_identity_verified  18488 non-null  object 
 14  host_response_rate      13933 non-null

In [10]:
train_pd.head()

,id,accommodates,amenities,bathrooms,bed_type,bedrooms,beds,cancellation_policy,city,cleaning_fee,...,longitude,name,neighbourhood,number_of_reviews,property_type,review_scores_rating,room_type,thumbnail_url,zipcode,y
0,0,6,"{TV,""Wireless Internet"",Kitchen,""Free parking ...",2.0,Real Bed,1.0,4.0,flexible,LA,t,...,-118.154761,The Penthouse,NaN,1,Apartment,60.0,Private room,NaN,90804,138.0
1,1,2,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",1.0,Real Bed,1.0,1.0,strict,DC,t,...,-76.978190,Guest Bedroom in Brookland,Brookland,9,House,100.0,Private room,https://a0.muscache.com/im/pictures/e4d8b51f-6...,20018,42.0
2,2,2,"{TV,Internet,""Wireless Internet"",Kitchen,""Indo...",2.0,Real Bed,1.0,1.0,strict,NYC,t,...,-73.926240,Clean Modern Room in Lux Apt 1 Block From J Train,Bushwick,27,Apartment,83.0,Private room,https://a0.muscache.com/im/pictures/5ffecc9b-d...,NaN,65.0
3,3,2,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",1.0,Real Bed,1.0,1.0,strict,SF,t,...,-122.411906,BEST views + reviews! 5/5 stars*****,Nob Hill,38,Apartment,95.0,Private room,NaN,94133,166.0
4,4,2,"{TV,Internet,""Wireless Internet"",""Air conditio...",1.0,Real Bed,1.0,1.0,strict,NYC,t,...,-73.974691,Charming 1-bedroom - UWS Manhattan,Upper West Side,5,Apartment,100.0,Entire home/apt,https://a0.muscache.com/im/pictures/92879730/5...,10024,165.0


In [11]:
test_pd.head()

,id,accommodates,amenities,bathrooms,bed_type,bedrooms,beds,cancellation_policy,city,cleaning_fee,...,latitude,longitude,name,neighbourhood,number_of_reviews,property_type,review_scores_rating,room_type,thumbnail_url,zipcode
0,0,6,"{TV,""Cable TV"",""Wireless Internet"",""Air condit...",2.0,Real Bed,2.0,2.0,strict,Boston,t,...,42.359278,-71.069962,Gorgeous 2BR/2BA Duplex in Beacon Hill,Beacon Hill,58,House,90.0,Entire home/apt,https://a0.muscache.com/im/pictures/7e4808b4-5...,02114
1,1,3,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",1.0,Real Bed,1.0,1.0,moderate,LA,t,...,34.084747,-118.367355,Luxury 1 Bedroom West Hollywood City Center,West Hollywood,4,Guesthouse,100.0,Entire home/apt,https://a0.muscache.com/im/pictures/5392fbd6-6...,90046
2,2,2,"{TV,""Wireless Internet"",""Air conditioning"",Kit...",1.0,Real Bed,0.0,1.0,flexible,NYC,f,...,40.720541,-73.959192,Bedroom with Patio in Prime Williamsburg Locat...,Williamsburg,0,Apartment,NaN,Private room,https://a0.muscache.com/im/pictures/544d3b89-d...,11249.0
3,3,4,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",1.0,Real Bed,1.0,2.0,strict,NYC,f,...,40.681117,-73.944091,Cozy apartment in Brooklyn,Bedford-Stuyvesant,0,Apartment,NaN,Entire home/apt,https://a0.muscache.com/im/pictures/26baf7ba-0...,11216
4,4,3,"{TV,Internet,""Wireless Internet"",""Air conditio...",1.5,Real Bed,1.0,2.0,strict,LA,t,...,34.150995,-118.409359,"Cozy, sunny, pet friendly loft/apt",NaN,6,Loft,92.0,Entire home/apt,https://a0.muscache.com/im/pictures/86107545/9...,91604


In [12]:
train_pd.corr()

,id,accommodates,bathrooms,bedrooms,beds,latitude,longitude,number_of_reviews,review_scores_rating,y
id,1.000000,0.005027,0.005182,0.002463,0.005736,-0.007358,-0.005771,0.004658,0.002307,-0.001129
accommodates,0.005027,1.000000,0.502278,0.708510,0.806654,-0.079395,-0.086365,0.042320,-0.017824,0.522330
bathrooms,0.005182,0.502278,1.000000,0.586366,0.524847,-0.135427,-0.132046,-0.040262,0.006698,0.456044
bedrooms,0.002463,0.708510,0.586366,1.000000,0.708230,-0.057502,-0.074027,-0.033761,0.009996,0.495830
beds,0.005736,0.806654,0.524847,0.708230,1.000000,-0.080307,-0.082434,0.030914,-0.027839,0.435603
latitude,-0.007358,-0.079395,-0.135427,-0.057502,-0.080307,1.000000,0.896581,-0.016439,-0.035133,-0.028940
longitude,-0.005771,-0.086365,-0.132046,-0.074027,-0.082434,0.896581,1.000000,-0.048660,-0.050019,-0.056236
number_of_reviews,0.004658,0.042320,-0.040262,-0.033761,0.030914,-0.016439,-0.048660,1.000000,0.012144,-0.068944
review_scores_rating,0.002307,-0.017824,0.006698,0.009996,-0.027839,-0.035133,-0.050019,0.012144,1.000000,0.065896
y,-0.001129,0.522330,0.456044,0.495830,0.435603,-0.028940,-0.056236,-0.068944,0.065896,1.000000


In [15]:
train_pd.corr()[train_pd.corr() > 0.3]

,id,accommodates,bathrooms,bedrooms,beds,latitude,longitude,number_of_reviews,review_scores_rating,y
id,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
accommodates,NaN,1.000000,0.502278,0.708510,0.806654,NaN,NaN,NaN,NaN,0.522330
bathrooms,NaN,0.502278,1.000000,0.586366,0.524847,NaN,NaN,NaN,NaN,0.456044
bedrooms,NaN,0.708510,0.586366,1.000000,0.708230,NaN,NaN,NaN,NaN,0.495830
beds,NaN,0.806654,0.524847,0.708230,1.000000,NaN,NaN,NaN,NaN,0.435603
latitude,NaN,NaN,NaN,NaN,NaN,1.000000,0.896581,NaN,NaN,NaN
longitude,NaN,NaN,NaN,NaN,NaN,0.896581,1.000000,NaN,NaN,NaN
number_of_reviews,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
review_scores_rating,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
y,NaN,0.522330,0.456044,0.495830,0.435603,NaN,NaN,NaN,NaN,1.000000


In [16]:
train_pd.corr()[train_pd.corr() < -0.3]

,id,accommodates,bathrooms,bedrooms,beds,latitude,longitude,number_of_reviews,review_scores_rating,y
id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
accommodates,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
bathrooms,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
bedrooms,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
beds,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
latitude,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
longitude,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
number_of_reviews,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
review_scores_rating,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
train_pd.corr()[['y']]

,y
id,-0.001129
accommodates,0.522330
bathrooms,0.456044
bedrooms,0.495830
beds,0.435603
latitude,-0.028940
longitude,-0.056236
number_of_reviews,-0.068944
review_scores_rating,0.065896
y,1.000000


In [34]:
train_pd.cleaning_fee.value_counts()

t    40821
f    14762
Name: cleaning_fee, dtype: int64

In [35]:
train_pd.host_has_profile_pic.value_counts()

t    55265
f      170
Name: host_has_profile_pic, dtype: int64

In [36]:
train_pd.host_identity_verified.value_counts()

t    37264
f    18171
Name: host_identity_verified, dtype: int64

In [37]:
train_pd.instant_bookable.value_counts()

f    41034
t    14549
Name: instant_bookable, dtype: int64

In [38]:
train_pd.property_type.value_counts()

Apartment             36826
House                 12326
Condominium            1983
Townhouse              1249
Loft                    914
Other                   448
Guesthouse              392
Bed & Breakfast         349
Bungalow                281
Villa                   131
Dorm                    112
Guest suite              96
Camper/RV                71
In-law                   60
Timeshare                55
Boat                     53
Hostel                   51
Boutique hotel           50
Cabin                    47
Serviced apartment       16
Tent                     14
Vacation home            11
Castle                    9
Yurt                      8
Treehouse                 7
Hut                       7
Earth House               4
Chalet                    4
Tipi                      3
Train                     2
Island                    1
Parking Space             1
Cave                      1
Casa particular           1
Name: property_type, dtype: int64

In [12]:
train_pd.property_type.value_counts()

Apartment             36826
House                 12326
Condominium            1983
Townhouse              1249
Loft                    914
Other                   448
Guesthouse              392
Bed & Breakfast         349
Bungalow                281
Villa                   131
Dorm                    112
Guest suite              96
Camper/RV                71
In-law                   60
Timeshare                55
Boat                     53
Hostel                   51
Boutique hotel           50
Cabin                    47
Serviced apartment       16
Tent                     14
Vacation home            11
Castle                    9
Yurt                      8
Treehouse                 7
Hut                       7
Chalet                    4
Earth House               4
Tipi                      3
Train                     2
Island                    1
Cave                      1
Parking Space             1
Casa particular           1
Name: property_type, dtype: int64

In [15]:
len(train_pd.property_type.unique())

34

In [16]:
train_pd.room_type.value_counts()

Entire home/apt    30939
Private room       23023
Shared room         1621
Name: room_type, dtype: int64

In [8]:
train_pd.amenities.value_counts()

{}                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  435
{"translation missing: en.hosting_amenity_49","translation missing: en.hosting_amenity_50"}                                                                                                                                                                                                                                                                                                                                     

In [9]:
train_pd.bed_type.value_counts()

Real Bed         53989
Futon              569
Pull-out Sofa      453
Airbed             364
Couch              208
Name: bed_type, dtype: int64

In [10]:
train_pd.cancellation_policy.value_counts()

strict             24318
flexible           16886
moderate           14288
super_strict_30       78
super_strict_60       13
Name: cancellation_policy, dtype: int64

In [11]:
train_pd.city.value_counts()

NYC        24326
LA         16828
SF          4768
DC          4259
Chicago     2807
Boston      2595
Name: city, dtype: int64

In [7]:
chv.check_miss_value(train_pd)

C:\Users\owner\codexa\600_SIGNATE\001_Assessment\check_miss_value.py:16: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  miss_table.coulmns = ['欠損数', '%']


,欠損数,%
id,0,0.000000
accommodates,0,0.000000
amenities,0,0.000000
bathrooms,147,0.264469
bed_type,0,0.000000
bedrooms,71,0.127737
beds,96,0.172715
cancellation_policy,0,0.000000
city,0,0.000000
cleaning_fee,0,0.000000
